#### "Smart Money" Tracker (Whale Watch)

##### Monitor high-net-worth "Whale" wallets to detect if they are preparing to sell (moving to exchanges) or hold (moving to cold storage).

In [4]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv

load_dotenv()
ETH_KEY = os.getenv('ETHERSCAN_API_KEY')

# Wintermute: One of the most active Market Makers in Jan 2026
WHALE_ADDR = "0xdbf5e9c5206d0db70a90108bf936da60221dc080"

#### The "Everything" Extractor

In [3]:
def fetch_all_tokens_v2(address, api_key):
    url = "https://api.etherscan.io/v2/api"
    params = {
        "chainid": "1",
        "module": "account",
        "action": "tokentx",     # Get ALL ERC-20 token transfers
        "address": address,
        "startblock": "0",
        "endblock": "99999999",
        "page": "1",
        "offset": "100",         # Pull 100 to ensure variety
        "sort": "desc",
        "apikey": api_key
    }
    
    response = requests.get(url, params=params).json()
    
    if response.get('status') == '1' and response.get('result'):
        df = pd.DataFrame(response['result'])
        # Data Cleaning
        df['datetime'] = pd.to_datetime(df['timeStamp'].astype(int), unit='s')
        df['amount'] = df.apply(lambda x: float(x['value']) / (10**int(x['tokenDecimal'])), axis=1)
        df['asset'] = df['tokenSymbol']
        
        # Check unique assets
        unique_assets = df['asset'].unique()
        print(f"Success! Found {len(df)} transfers across {len(unique_assets)} unique assets.")
        print(f"Assets found: {', '.join(unique_assets)}")
        
        return df[['datetime', 'from', 'to', 'amount', 'asset', 'hash']]
    else:
        print(f"API Warning: {response.get('message', 'No data found')}")
        return pd.DataFrame()

# run the extraction
whale_df = fetch_all_tokens_v2(WHALE_ADDR, ETH_KEY)

Success! Found 100 transfers across 37 unique assets.
Assets found: 𝑬𝑻𝑯, eHYPER, BUIDL, BrainETH, Openx, FDUSD, wkeyDAO, USDC, GRVT, NVIDIA, OscarAI, Lighter, ETH LIFE, wPERP, EdgeX, PEPE, USDT, ASTER, NOVA, GRT, PROXIMA, NVIDIA AI, SPX, E AI, DRV, HYPER, WLFI, TESLA AI, GROK4 AI, POL, YOKAI, GPT-5, KTTY, YFED, ENA, sENA, QUANT


#### The Entity Mapping Dictionary

In [5]:
CEX_MAP = {
    # Exchanges (CEX)
    "0x28c6c06290cc3f9517c3910028e7244017a97437": "Binance: Hot Wallet",
    "0x71660c4dbd25f0df0f2148d8871a536653396b72": "Coinbase: Deposit",
    "0x503828976d22510aad0201ac7ec88293211d23da": "Gemini: Institutional",
    "0xab5801a7d398351b8be11c439e05c5b3259aec9b": "Vitalik Buterin (Vb2)",
    
    # Market Makers / Whales
    "0xdbf5e9c5206d0db70a90108bf936da60221dc080": "Wintermute (Whale)",
    "0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2": "Wrapped Ether (WETH)"
}

def label_entity(address):
    return CEX_MAP.get(address.lower(), "Private Wallet")

# Apply the labels to our dataframe
whale_df['from_entity'] = whale_df['from'].apply(label_entity)
whale_df['to_entity'] = whale_df['to'].apply(label_entity)

#### The Intent Logic (Sell Pressure vs. Accumulation)

- Applying an "Intent Rule" based on the direction of funds.

- We are building an Indicator.

    - Inflow to CEX = Potential Selling (Price Risk).
    
    - Outflow from CEX = Potential Buying/Holding (Price Support)

In [6]:
def classify_intent(row):
    # Rule 1: Moving TO a known Exchange
    if "Binance" in row['to_entity'] or "Coinbase" in row['to_entity'] or "Gemini" in row['to_entity']:
        return "EXCHANGE INFLOW (Sell Risk)"
    
    # Rule 2: Moving FROM a known Exchange
    elif "Binance" in row['from_entity'] or "Coinbase" in row['from_entity'] or "Gemini" in row['from_entity']:
        return "EXCHANGE OUTFLOW (Accumulation)"
    
    # Rule 3: Internal/Private Moves
    return "PRIVATE/INTERNAL MOVE"

whale_df['intent'] = whale_df.apply(classify_intent, axis=1)

# Preview our classified signals
whale_df[['datetime', 'amount', 'asset', 'from_entity', 'to_entity', 'intent']].head(10)

,datetime,amount,asset,from_entity,to_entity,intent
0,2026-01-28 05:29:59,6.477146e+03,𝑬𝑻𝑯,Wintermute (Whale),Private Wallet,PRIVATE/INTERNAL MOVE
1,2026-01-28 05:29:47,1.000000e-02,𝑬𝑻𝑯,Wintermute (Whale),Private Wallet,PRIVATE/INTERNAL MOVE
2,2026-01-18 13:12:35,9.600000e+10,eHYPER,Private Wallet,Wintermute (Whale),PRIVATE/INTERNAL MOVE
3,2025-12-17 16:06:47,2.000000e+01,BUIDL,Wintermute (Whale),Private Wallet,PRIVATE/INTERNAL MOVE
4,2025-12-03 21:48:23,3.550000e+11,BrainETH,Private Wallet,Wintermute (Whale),PRIVATE/INTERNAL MOVE
5,2025-11-10 05:13:59,6.000000e+12,Openx,Private Wallet,Wintermute (Whale),PRIVATE/INTERNAL MOVE
6,2025-10-31 10:14:47,5.834921e+05,FDUSD,Wintermute (Whale),Private Wallet,PRIVATE/INTERNAL MOVE
7,2025-10-31 10:06:35,5.834921e+05,FDUSD,Private Wallet,Wintermute (Whale),PRIVATE/INTERNAL MOVE
8,2025-10-22 02:51:59,1.000000e+14,wkeyDAO,Private Wallet,Wintermute (Whale),PRIVATE/INTERNAL MOVE
9,2025-10-22 02:48:59,2.000000e+00,USDC,Wintermute (Whale),Private Wallet,PRIVATE/INTERNAL MOVE


#### Global Display Fix

In [7]:
# Suppressing scientific notation globally
# To make massive token volumes readable for humans
import pandas as pd

pd.set_option('display.float_format', lambda x: '{:,.2f}'.format(x))

# Preview the change
display(whale_df.tail())

,datetime,from,to,amount,asset,hash,from_entity,to_entity,intent
95,2025-07-31 08:13:59,0xa511654f52cc00136cd568ae117d73aaa3ea7d25,0xdbf5e9c5206d0db70a90108bf936da60221dc080,"22,000,000,000.00",QUANT,0xc45adfd1532037818cbf14de56b160dc9dd992cca886...,Private Wallet,Wintermute (Whale),PRIVATE/INTERNAL MOVE
96,2025-07-31 08:07:47,0xa511654f52cc00136cd568ae117d73aaa3ea7d25,0xdbf5e9c5206d0db70a90108bf936da60221dc080,"22,000,000,000.00",QUANT,0xa73945d18f58a4675c03c4e12ec703b4f59897d56de4...,Private Wallet,Wintermute (Whale),PRIVATE/INTERNAL MOVE
97,2025-07-31 08:05:47,0xa511654f52cc00136cd568ae117d73aaa3ea7d25,0xdbf5e9c5206d0db70a90108bf936da60221dc080,"22,000,000,000.00",QUANT,0x60368b84b584b3c44e3db789241879c995e644273d26...,Private Wallet,Wintermute (Whale),PRIVATE/INTERNAL MOVE
98,2025-07-27 13:24:11,0x9bd50ed8a0d183396caaf463f6c89e9deb643537,0xdbf5e9c5206d0db70a90108bf936da60221dc080,"332,145,678,963,258.75",TESLA AI,0x9bf827816de511910d09db9e7f6a80ed91e0df605a1d...,Private Wallet,Wintermute (Whale),PRIVATE/INTERNAL MOVE
99,2025-07-27 13:19:59,0x9bd50ed8a0d183396caaf463f6c89e9deb643537,0xdbf5e9c5206d0db70a90108bf936da60221dc080,"11,000,000,000,000.00",TESLA AI,0x1c77ad466b2bc79f2f40f8f965bda0f7787ca1491987...,Private Wallet,Wintermute (Whale),PRIVATE/INTERNAL MOVE
